In [ ]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,StackingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report,accuracy_score
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import pickle
import random

df = pd.read_csv('/content/drive/MyDrive/Road.csv')
df

,Time,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,...,Vehicle_movement,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Work_of_casuality,Fitness_of_casuality,Pedestrian_movement,Cause_of_accident,Accident_severity
0,17:02:00,Monday,18-30,Male,Above high school,Employee,1-2yr,Automobile,Owner,Above 10yr,...,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Moving Backward,Slight Injury
1,17:02:00,Monday,31-50,Male,Junior high school,Employee,Above 10yr,Public (> 45 seats),Owner,5-10yrs,...,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury
2,17:02:00,Monday,18-30,Male,Junior high school,Employee,1-2yr,Lorry (41?100Q),Owner,NaN,...,Going straight,Driver or rider,Male,31-50,3,Driver,NaN,Not a Pedestrian,Changing lane to the left,Serious Injury
3,1:06:00,Sunday,18-30,Male,Junior high school,Employee,5-10yr,Public (> 45 seats),Governmental,NaN,...,Going straight,Pedestrian,Female,18-30,3,Driver,Normal,Not a Pedestrian,Changing lane to the right,Slight Injury
4,1:06:00,Sunday,18-30,Male,Junior high school,Employee,2-5yr,NaN,Owner,5-10yrs,...,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12311,16:15:00,Wednesday,31-50,Male,NaN,Employee,2-5yr,Lorry (11?40Q),Owner,NaN,...,Going straight,na,na,na,na,Driver,Normal,Not a Pedestrian,No distancing,Slight Injury
12312,18:00:00,Sunday,Unknown,Male,Elementary school,Employee,5-10yr,Automobile,Owner,NaN,...,Other,na,na,na,na,Driver,Normal,Not a Pedestrian,No distancing,Slight Injury
12313,13:55:00,Sunday,Over 51,Male,Junior high school,Employee,5-10yr,Bajaj,Owner,2-5yrs,...,Other,Driver or rider,Male,31-50,3,Driver,Normal,Not a Pedestrian,Changing lane to the right,Serious Injury
12314,13:55:00,Sunday,18-30,Female,Junior high school,Employee,Above 10yr,Lorry (41?100Q),Owner,2-5yrs,...,Other,na,na,na,na,Driver,Normal,Not a Pedestrian,Driving under the influence of drugs,Slight Injury


Replace the 'na' and 'Unknown' ton Nan:

In [ ]:
df.replace(['na','Unknown','Unknown or other','unknown','other','Other'],np.nan,inplace=True)
df

,Time,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,...,Vehicle_movement,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Work_of_casuality,Fitness_of_casuality,Pedestrian_movement,Cause_of_accident,Accident_severity
0,17:02:00,Monday,18-30,Male,Above high school,Employee,1-2yr,Automobile,Owner,Above 10yr,...,Going straight,NaN,NaN,NaN,NaN,NaN,NaN,Not a Pedestrian,Moving Backward,Slight Injury
1,17:02:00,Monday,31-50,Male,Junior high school,Employee,Above 10yr,Public (> 45 seats),Owner,5-10yrs,...,Going straight,NaN,NaN,NaN,NaN,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury
2,17:02:00,Monday,18-30,Male,Junior high school,Employee,1-2yr,Lorry (41?100Q),Owner,NaN,...,Going straight,Driver or rider,Male,31-50,3,Driver,NaN,Not a Pedestrian,Changing lane to the left,Serious Injury
3,1:06:00,Sunday,18-30,Male,Junior high school,Employee,5-10yr,Public (> 45 seats),Governmental,NaN,...,Going straight,Pedestrian,Female,18-30,3,Driver,Normal,Not a Pedestrian,Changing lane to the right,Slight Injury
4,1:06:00,Sunday,18-30,Male,Junior high school,Employee,2-5yr,NaN,Owner,5-10yrs,...,Going straight,NaN,NaN,NaN,NaN,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12311,16:15:00,Wednesday,31-50,Male,NaN,Employee,2-5yr,Lorry (11?40Q),Owner,NaN,...,Going straight,NaN,NaN,NaN,NaN,Driver,Normal,Not a Pedestrian,No distancing,Slight Injury
12312,18:00:00,Sunday,NaN,Male,Elementary school,Employee,5-10yr,Automobile,Owner,NaN,...,NaN,NaN,NaN,NaN,NaN,Driver,Normal,Not a Pedestrian,No distancing,Slight Injury
12313,13:55:00,Sunday,Over 51,Male,Junior high school,Employee,5-10yr,Bajaj,Owner,2-5yrs,...,NaN,Driver or rider,Male,31-50,3,Driver,Normal,Not a Pedestrian,Changing lane to the right,Serious Injury
12314,13:55:00,Sunday,18-30,Female,Junior high school,Employee,Above 10yr,Lorry (41?100Q),Owner,2-5yrs,...,NaN,NaN,NaN,NaN,NaN,Driver,Normal,Not a Pedestrian,Driving under the influence of drugs,Slight Injury


**Cleaning the data**

In [ ]:
df.isna().sum() # finding the null values

,0
Time,0
Day_of_week,0
Age_band_of_driver,1548
Sex_of_driver,178
Educational_level,841
Vehicle_driver_relation,716
Driving_experience,862
Type_of_vehicle,2158
Owner_of_vehicle,504
Service_year_of_vehicle,6811


In [ ]:
df.dtypes  # checking the types of columns

,0
Time,object
Day_of_week,object
Age_band_of_driver,object
Sex_of_driver,object
Educational_level,object
Vehicle_driver_relation,object
Driving_experience,object
Type_of_vehicle,object
Owner_of_vehicle,object
Service_year_of_vehicle,object


In [ ]:
# filling the null values
for col in df.columns:
  if df[col].dtypes=='object'and df[col].isna().any():
    df[col] = df[col].fillna(df[col].mode()[0])
df

,Time,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,...,Vehicle_movement,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Work_of_casuality,Fitness_of_casuality,Pedestrian_movement,Cause_of_accident,Accident_severity
0,17:02:00,Monday,18-30,Male,Above high school,Employee,1-2yr,Automobile,Owner,Above 10yr,...,Going straight,Driver or rider,Male,18-30,3,Driver,Normal,Not a Pedestrian,Moving Backward,Slight Injury
1,17:02:00,Monday,31-50,Male,Junior high school,Employee,Above 10yr,Public (> 45 seats),Owner,5-10yrs,...,Going straight,Driver or rider,Male,18-30,3,Driver,Normal,Not a Pedestrian,Overtaking,Slight Injury
2,17:02:00,Monday,18-30,Male,Junior high school,Employee,1-2yr,Lorry (41?100Q),Owner,2-5yrs,...,Going straight,Driver or rider,Male,31-50,3,Driver,Normal,Not a Pedestrian,Changing lane to the left,Serious Injury
3,1:06:00,Sunday,18-30,Male,Junior high school,Employee,5-10yr,Public (> 45 seats),Governmental,2-5yrs,...,Going straight,Pedestrian,Female,18-30,3,Driver,Normal,Not a Pedestrian,Changing lane to the right,Slight Injury
4,1:06:00,Sunday,18-30,Male,Junior high school,Employee,2-5yr,Automobile,Owner,5-10yrs,...,Going straight,Driver or rider,Male,18-30,3,Driver,Normal,Not a Pedestrian,Overtaking,Slight Injury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12311,16:15:00,Wednesday,31-50,Male,Junior high school,Employee,2-5yr,Lorry (11?40Q),Owner,2-5yrs,...,Going straight,Driver or rider,Male,18-30,3,Driver,Normal,Not a Pedestrian,No distancing,Slight Injury
12312,18:00:00,Sunday,18-30,Male,Elementary school,Employee,5-10yr,Automobile,Owner,2-5yrs,...,Going straight,Driver or rider,Male,18-30,3,Driver,Normal,Not a Pedestrian,No distancing,Slight Injury
12313,13:55:00,Sunday,Over 51,Male,Junior high school,Employee,5-10yr,Bajaj,Owner,2-5yrs,...,Going straight,Driver or rider,Male,31-50,3,Driver,Normal,Not a Pedestrian,Changing lane to the right,Serious Injury
12314,13:55:00,Sunday,18-30,Female,Junior high school,Employee,Above 10yr,Lorry (41?100Q),Owner,2-5yrs,...,Going straight,Driver or rider,Male,18-30,3,Driver,Normal,Not a Pedestrian,Driving under the influence of drugs,Slight Injury


In [ ]:
df.isna().sum()

,0
Time,0
Day_of_week,0
Age_band_of_driver,0
Sex_of_driver,0
Educational_level,0
Vehicle_driver_relation,0
Driving_experience,0
Type_of_vehicle,0
Owner_of_vehicle,0
Service_year_of_vehicle,0


Cleaning the corrupted text

In [ ]:
df['Casualty_severity'].unique()

array(['3', '2', '1'], dtype=object)

In [ ]:
severity = {'1':'Fatal','2':'Serious','3':'Slight'}
df['Casualty_severity'] = df['Casualty_severity'].map(severity)

In [ ]:
# Convert string to datetime
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S')

# Extract hour
df['Hour'] = df['Time'].dt.hour

def categorize_time(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df['Time_of_Day'] = df['Hour'].apply(categorize_time)

In [ ]:
df['Time_of_Day'].unique()

array(['Evening', 'Night', 'Afternoon', 'Morning'], dtype=object)

In [ ]:
df.drop(['Time','Hour'],axis=1,inplace=True)
df

,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,Defect_of_vehicle,...,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Work_of_casuality,Fitness_of_casuality,Pedestrian_movement,Cause_of_accident,Accident_severity,Time_of_Day
0,Monday,18-30,Male,Above high school,Employee,1-2yr,Automobile,Owner,Above 10yr,No defect,...,Driver or rider,Male,18-30,Slight,Driver,Normal,Not a Pedestrian,Moving Backward,Slight Injury,Evening
1,Monday,31-50,Male,Junior high school,Employee,Above 10yr,Public (> 45 seats),Owner,5-10yrs,No defect,...,Driver or rider,Male,18-30,Slight,Driver,Normal,Not a Pedestrian,Overtaking,Slight Injury,Evening
2,Monday,18-30,Male,Junior high school,Employee,1-2yr,Lorry (41?100Q),Owner,2-5yrs,No defect,...,Driver or rider,Male,31-50,Slight,Driver,Normal,Not a Pedestrian,Changing lane to the left,Serious Injury,Evening
3,Sunday,18-30,Male,Junior high school,Employee,5-10yr,Public (> 45 seats),Governmental,2-5yrs,No defect,...,Pedestrian,Female,18-30,Slight,Driver,Normal,Not a Pedestrian,Changing lane to the right,Slight Injury,Night
4,Sunday,18-30,Male,Junior high school,Employee,2-5yr,Automobile,Owner,5-10yrs,No defect,...,Driver or rider,Male,18-30,Slight,Driver,Normal,Not a Pedestrian,Overtaking,Slight Injury,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12311,Wednesday,31-50,Male,Junior high school,Employee,2-5yr,Lorry (11?40Q),Owner,2-5yrs,No defect,...,Driver or rider,Male,18-30,Slight,Driver,Normal,Not a Pedestrian,No distancing,Slight Injury,Afternoon
12312,Sunday,18-30,Male,Elementary school,Employee,5-10yr,Automobile,Owner,2-5yrs,No defect,...,Driver or rider,Male,18-30,Slight,Driver,Normal,Not a Pedestrian,No distancing,Slight Injury,Evening
12313,Sunday,Over 51,Male,Junior high school,Employee,5-10yr,Bajaj,Owner,2-5yrs,No defect,...,Driver or rider,Male,31-50,Slight,Driver,Normal,Not a Pedestrian,Changing lane to the right,Serious Injury,Afternoon
12314,Sunday,18-30,Female,Junior high school,Employee,Above 10yr,Lorry (41?100Q),Owner,2-5yrs,No defect,...,Driver or rider,Male,18-30,Slight,Driver,Normal,Not a Pedestrian,Driving under the influence of drugs,Slight Injury,Afternoon


In [ ]:
df['Age_band_of_casualty'].unique()

array(['18-30', '31-50', 'Under 18', 'Over 51', '5'], dtype=object)

In [ ]:
df['Age_band_of_casualty'] =df['Age_band_of_casualty'].replace('5','Under 18')
df

,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,Defect_of_vehicle,...,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Work_of_casuality,Fitness_of_casuality,Pedestrian_movement,Cause_of_accident,Accident_severity,Time_of_Day
0,Monday,18-30,Male,Above high school,Employee,1-2yr,Automobile,Owner,Above 10yr,No defect,...,Driver or rider,Male,18-30,Slight,Driver,Normal,Not a Pedestrian,Moving Backward,Slight Injury,Evening
1,Monday,31-50,Male,Junior high school,Employee,Above 10yr,Public (> 45 seats),Owner,5-10yrs,No defect,...,Driver or rider,Male,18-30,Slight,Driver,Normal,Not a Pedestrian,Overtaking,Slight Injury,Evening
2,Monday,18-30,Male,Junior high school,Employee,1-2yr,Lorry (41?100Q),Owner,2-5yrs,No defect,...,Driver or rider,Male,31-50,Slight,Driver,Normal,Not a Pedestrian,Changing lane to the left,Serious Injury,Evening
3,Sunday,18-30,Male,Junior high school,Employee,5-10yr,Public (> 45 seats),Governmental,2-5yrs,No defect,...,Pedestrian,Female,18-30,Slight,Driver,Normal,Not a Pedestrian,Changing lane to the right,Slight Injury,Night
4,Sunday,18-30,Male,Junior high school,Employee,2-5yr,Automobile,Owner,5-10yrs,No defect,...,Driver or rider,Male,18-30,Slight,Driver,Normal,Not a Pedestrian,Overtaking,Slight Injury,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12311,Wednesday,31-50,Male,Junior high school,Employee,2-5yr,Lorry (11?40Q),Owner,2-5yrs,No defect,...,Driver or rider,Male,18-30,Slight,Driver,Normal,Not a Pedestrian,No distancing,Slight Injury,Afternoon
12312,Sunday,18-30,Male,Elementary school,Employee,5-10yr,Automobile,Owner,2-5yrs,No defect,...,Driver or rider,Male,18-30,Slight,Driver,Normal,Not a Pedestrian,No distancing,Slight Injury,Evening
12313,Sunday,Over 51,Male,Junior high school,Employee,5-10yr,Bajaj,Owner,2-5yrs,No defect,...,Driver or rider,Male,31-50,Slight,Driver,Normal,Not a Pedestrian,Changing lane to the right,Serious Injury,Afternoon
12314,Sunday,18-30,Female,Junior high school,Employee,Above 10yr,Lorry (41?100Q),Owner,2-5yrs,No defect,...,Driver or rider,Male,18-30,Slight,Driver,Normal,Not a Pedestrian,Driving under the influence of drugs,Slight Injury,Afternoon


In [ ]:
# Define a mapping of corrupted → correct text
pedestrian_mapping = {
    'Not a Pedestrian': 'Not a Pedestrian',
    "Crossing from driver's nearside": "Crossing from driver's nearside",
    'Crossing from nearside - masked by parked or statioNot a Pedestrianry vehicle':
        'Crossing from nearside - masked by parked or stationary vehicle',
    'Crossing from offside - masked by  parked or statioNot a Pedestrianry vehicle':
        'Crossing from offside - masked by parked or stationary vehicle',
    'In carriageway, statioNot a Pedestrianry - not crossing  (standing or playing)':
        'In carriageway, stationary - not crossing (standing or playing)',
    'Walking along in carriageway, back to traffic':
        'Walking along in carriageway, back to traffic',
    'Walking along in carriageway, facing traffic':
        'Walking along in carriageway, facing traffic',
    'In carriageway, statioNot a Pedestrianry - not crossing  (standing or playing) - masked by parked or statioNot a Pedestrianry vehicle':
        'In carriageway, stationary - not crossing (standing or playing) - masked by parked or stationary vehicle'
}

# Apply the mapping safely
df['Pedestrian_movement'] = df['Pedestrian_movement'].replace(pedestrian_mapping)

# (Optional) Verify if everything was replaced correctly
print(df['Pedestrian_movement'].unique())


['Not a Pedestrian' "Crossing from driver's nearside"
 'Crossing from nearside - masked by parked or stationary vehicle'
 'Crossing from offside - masked by parked or stationary vehicle'
 'In carriageway, stationary - not crossing (standing or playing)'
 'Walking along in carriageway, back to traffic'
 'Walking along in carriageway, facing traffic'
 'In carriageway, stationary - not crossing (standing or playing) - masked by parked or stationary vehicle']


In [ ]:
# Area_accident_occured: strip spaces and fix merged value
df['Area_accident_occured'] = df['Area_accident_occured'].str.strip()
df['Area_accident_occured'] = df['Area_accident_occured'].replace({
    'Rural village areasOffice areas': 'Rural village areas'
})

# Road_allignment: fix incomplete text
df['Road_allignment'] = df['Road_allignment'].replace({
    'Tangent road with mountainous terrain and': 'Tangent road with mountainous terrain'
})

# Fitness_of_casuality: remove duplicate entry
df['Fitness_of_casuality'] = df['Fitness_of_casuality'].replace({
    'NormalNormal': 'Normal'
})

# Type_of_vehicle: fix weird symbols
df['Type_of_vehicle'] = df['Type_of_vehicle'].replace({
    'Lorry (11?40Q)': 'Lorry (11–40Q)',
    'Lorry (41?100Q)': 'Lorry (41–100Q)'
})

print("All corrupted categorical values have been cleaned!")

All corrupted categorical values have been cleaned!


**Encoding the Categorical values**

In [ ]:
np.random.seed(42)
random.seed(42)

In [ ]:
encoders = {}
categories = [col for col in df.columns if df[col].dtypes == 'object']

for col in categories:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

df

,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,Defect_of_vehicle,...,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Work_of_casuality,Fitness_of_casuality,Pedestrian_movement,Cause_of_accident,Accident_severity,Time_of_Day
0,1,0,1,0,0,0,0,2,3,2,...,0,1,0,2,0,2,5,9,2,1
1,1,1,1,4,0,3,10,2,2,2,...,0,1,0,2,0,2,5,15,2,1
2,1,0,1,4,0,0,5,2,1,2,...,0,1,1,2,0,2,5,0,1,1
3,3,0,1,4,0,2,10,0,1,2,...,2,0,0,2,0,2,5,1,2,3
4,3,0,1,4,0,1,0,2,2,2,...,0,1,0,2,0,2,5,15,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12311,6,1,1,4,0,1,4,2,1,2,...,0,1,0,2,0,2,5,10,2,0
12312,3,0,1,1,0,2,0,2,1,2,...,0,1,0,2,0,2,5,10,2,1
12313,3,2,1,4,0,2,1,2,1,2,...,0,1,1,2,0,2,5,1,1,0
12314,3,0,0,4,0,3,5,2,1,2,...,0,1,0,2,0,2,5,5,2,0


In [ ]:
X = df.drop(['Accident_severity'],axis=1).values
y = df['Accident_severity']

In [ ]:
# Scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[0.16666667, 0.        , 1.        , ..., 0.71428571, 0.52941176,
        0.33333333],
       [0.16666667, 0.33333333, 1.        , ..., 0.71428571, 0.88235294,
        0.33333333],
       [0.16666667, 0.        , 1.        , ..., 0.71428571, 0.        ,
        0.33333333],
       ...,
       [0.5       , 0.66666667, 1.        , ..., 0.71428571, 0.05882353,
        0.        ],
       [0.5       , 0.        , 0.        , ..., 0.71428571, 0.29411765,
        0.        ],
       [0.5       , 0.        , 1.        , ..., 0.14285714, 0.05882353,
        0.        ]])

**Splitting the data as Training and Testing data**

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_scaled,y,test_size=0.3,random_state = 42)

In [ ]:
models = {'KNN': KNeighborsClassifier(),'GaussianNB':GaussianNB(),'SVC':SVC(),'DecisionTree':DecisionTreeClassifier(random_state=42),'RandomForest':RandomForestClassifier(random_state=42)}
for model_name,model in models.items():
  print(model_name)
  model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  print(classification_report(y_test,y_pred))

KNN
              precision    recall  f1-score   support

           0       0.33      0.02      0.04        52
           1       0.22      0.03      0.06       552
           2       0.84      0.98      0.90      3091

    accuracy                           0.82      3695
   macro avg       0.46      0.34      0.33      3695
weighted avg       0.74      0.82      0.76      3695

GaussianNB
              precision    recall  f1-score   support

           0       0.01      0.92      0.03        52
           1       0.17      0.02      0.03       552
           2       0.84      0.07      0.14      3091

    accuracy                           0.08      3695
   macro avg       0.34      0.34      0.06      3695
weighted avg       0.73      0.08      0.12      3695

SVC
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        52
           1       0.00      0.00      0.00       552
           2       0.84      1.00      0.91      3091

 

In [ ]:
model_boosting = {'GradientBoosting':GradientBoostingClassifier(random_state=42),'AdaBoost':AdaBoostClassifier(random_state=42),'XGBoost':XGBClassifier(random_state=42)}
for model_name,model in model_boosting.items():
  print(model_name)
  model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  print(classification_report(y_test,y_pred))

GradientBoosting
              precision    recall  f1-score   support

           0       0.20      0.02      0.04        52
           1       0.70      0.05      0.09       552
           2       0.84      1.00      0.91      3091

    accuracy                           0.84      3695
   macro avg       0.58      0.35      0.35      3695
weighted avg       0.81      0.84      0.78      3695

AdaBoost
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        52
           1       0.00      0.00      0.00       552
           2       0.84      1.00      0.91      3091

    accuracy                           0.84      3695
   macro avg       0.28      0.33      0.30      3695
weighted avg       0.70      0.84      0.76      3695

XGBoost
              precision    recall  f1-score   support

           0       0.29      0.04      0.07        52
           1       0.46      0.08      0.14       552
           2       0.85      0.98      0.

In [ ]:
df['Accident_severity'].value_counts()

,count
Accident_severity,
2,10415
1,1743
0,158


In [ ]:
y_train.value_counts()

,count
Accident_severity,
2,7324
1,1191
0,106


there is a need for resampling the because of the minority classes 0 and 1

**Resampling using SMOTE**

In [ ]:
smote = SMOTE(random_state=42)
X_train_res,y_train_res = smote.fit_resample(X_train,y_train)

In [ ]:
y_train_res.value_counts()

,count
Accident_severity,
2,7324
1,7324
0,7324


In [ ]:
models = {'KNN': KNeighborsClassifier(),'GaussianNB':GaussianNB(),'SVC':SVC(),
          'DescisionTree':DecisionTreeClassifier(random_state=42),'RandomForest':RandomForestClassifier(random_state=42)}
for model_name,model in models.items():
  print(model_name)
  model.fit(X_train_res,y_train_res)
  y_pred = model.predict(X_test)
  print(classification_report(y_test,y_pred))

KNN
              precision    recall  f1-score   support

           0       0.03      0.19      0.05        52
           1       0.15      0.40      0.22       552
           2       0.84      0.52      0.64      3091

    accuracy                           0.50      3695
   macro avg       0.34      0.37      0.30      3695
weighted avg       0.73      0.50      0.57      3695

GaussianNB
              precision    recall  f1-score   support

           0       0.01      0.92      0.03        52
           1       0.23      0.03      0.05       552
           2       0.86      0.09      0.16      3091

    accuracy                           0.09      3695
   macro avg       0.37      0.35      0.08      3695
weighted avg       0.75      0.09      0.14      3695

SVC
              precision    recall  f1-score   support

           0       0.04      0.12      0.06        52
           1       0.17      0.30      0.22       552
           2       0.85      0.70      0.77      3091

 

In [ ]:
models_boosting = {'GradientBoosting':GradientBoostingClassifier(random_state=42),'AdaBoost':AdaBoostClassifier(random_state=42),'XGBoost':XGBClassifier(random_state=42)}
for model_name,model in models_boosting.items():
  print(model_name)
  model.fit(X_train_res,y_train_res)
  y_pred = model.predict(X_test)
  print(classification_report(y_test,y_pred))

GradientBoosting
              precision    recall  f1-score   support

           0       0.14      0.13      0.14        52
           1       0.45      0.04      0.07       552
           2       0.84      0.98      0.91      3091

    accuracy                           0.83      3695
   macro avg       0.48      0.38      0.37      3695
weighted avg       0.77      0.83      0.77      3695

AdaBoost
              precision    recall  f1-score   support

           0       0.06      0.33      0.10        52
           1       0.21      0.25      0.23       552
           2       0.86      0.76      0.81      3091

    accuracy                           0.68      3695
   macro avg       0.38      0.45      0.38      3695
weighted avg       0.75      0.68      0.71      3695

XGBoost
              precision    recall  f1-score   support

           0       0.14      0.02      0.03        52
           1       0.48      0.08      0.13       552
           2       0.85      0.99      0.

We can we that RandomForest and GradientBoost and XGBoost gives good accuracy irrespective of the very low recall and precision for minority classes.

**FEATURE SELECTION AND HYPERPARAMETER TUNING**

In [ ]:
from sklearn.feature_selection import SelectFromModel
xgbmodel = XGBClassifier(random_state=42)
xgbmodel.fit(X_train_res,y_train_res)

importances = xgbmodel.feature_importances_
feature_importances = pd.DataFrame({
    'Feature': df.drop(['Accident_severity'],axis=1).columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
print(feature_importances)

selector = SelectFromModel(xgbmodel,threshold=0.021921,prefit=True)
selected_indices = selector.get_support(indices=True)
selected_features = df.drop(['Accident_severity'], axis=1).columns[selected_indices]
print("Selected Features:\n",selected_features)
print("No. of features selected:",len(selected_features))

                        Feature  Importance
20         Number_of_casualties    0.125054
19  Number_of_vehicles_involved    0.102837
5            Driving_experience    0.061614
1            Age_band_of_driver    0.056366
16             Light_conditions    0.048635
13            Types_of_Junction    0.043581
15      Road_surface_conditions    0.034809
26            Work_of_casuality    0.033899
29            Cause_of_accident    0.031150
14            Road_surface_type    0.031073
25            Casualty_severity    0.030476
0                   Day_of_week    0.030396
11             Lanes_or_Medians    0.029599
4       Vehicle_driver_relation    0.027788
6               Type_of_vehicle    0.027614
30                  Time_of_Day    0.026978
21             Vehicle_movement    0.025216
22               Casualty_class    0.024499
2                 Sex_of_driver    0.023078
10        Area_accident_occured    0.023076
8       Service_year_of_vehicle    0.021920
18            Type_of_collision 

In [ ]:
X_train_selected = selector.transform(X_train_res)
X_test_selected = selector.transform(X_test)

X_train_selected = pd.DataFrame(X_train_selected, columns=selected_features)
X_test_selected = pd.DataFrame(X_test_selected, columns=selected_features)

In [ ]:
models = {'KNN': KNeighborsClassifier(),'GaussianNB':GaussianNB(),'SVC':SVC(),
          'DescisionTree':DecisionTreeClassifier(random_state=42),'RandomForest':RandomForestClassifier(random_state=42)}
for model_name,model in models.items():
  print(model_name)
  model.fit(X_train_selected,y_train_res)
  y_pred_selected = model.predict(X_test_selected)
  print(classification_report(y_test,y_pred_selected))

KNN
              precision    recall  f1-score   support

           0       0.03      0.17      0.05        52
           1       0.16      0.40      0.23       552
           2       0.85      0.55      0.67      3091

    accuracy                           0.53      3695
   macro avg       0.35      0.38      0.32      3695
weighted avg       0.73      0.53      0.60      3695

GaussianNB
              precision    recall  f1-score   support

           0       0.02      0.69      0.04        52
           1       0.16      0.21      0.18       552
           2       0.84      0.31      0.45      3091

    accuracy                           0.30      3695
   macro avg       0.34      0.40      0.22      3695
weighted avg       0.73      0.30      0.40      3695

SVC
              precision    recall  f1-score   support

           0       0.06      0.27      0.09        52
           1       0.17      0.33      0.22       552
           2       0.85      0.66      0.74      3091

 

In [ ]:
models_boosting = {'GradientBoosting':GradientBoostingClassifier(random_state=42),'AdaBoost':AdaBoostClassifier(random_state=42),'XGBoost':XGBClassifier(random_state=42)}
for model_name,model in models_boosting.items():
  print(model_name)
  model.fit(X_train_selected,y_train_res)
  y_pred_selected = model.predict(X_test_selected)
  print(classification_report(y_test,y_pred_selected))

GradientBoosting
              precision    recall  f1-score   support

           0       0.16      0.15      0.16        52
           1       0.39      0.04      0.07       552
           2       0.84      0.98      0.90      3091

    accuracy                           0.83      3695
   macro avg       0.46      0.39      0.38      3695
weighted avg       0.76      0.83      0.77      3695

AdaBoost
              precision    recall  f1-score   support

           0       0.07      0.31      0.11        52
           1       0.19      0.26      0.22       552
           2       0.85      0.75      0.80      3091

    accuracy                           0.67      3695
   macro avg       0.37      0.44      0.38      3695
weighted avg       0.74      0.67      0.70      3695

XGBoost
              precision    recall  f1-score   support

           0       0.33      0.06      0.10        52
           1       0.44      0.09      0.15       552
           2       0.85      0.98      0.

Hyperparameter Tuning using Grid Search and Random Search

In [ ]:
# Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
models = {
    'SVC': SVC(),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False,random_state=42)
}
param_grids = {
    'SVC': {
        'clf__C': [0.1, 1, 10],
        'clf__kernel': ['linear', 'rbf'],
        'clf__gamma': ['scale', 'auto']
    },
    'DecisionTree': {
        'clf__criterion': ['gini', 'entropy'],
        'clf__max_depth': [None, 10, 20],
        'clf__min_samples_split': [2, 5, 10]
    },
    'RandomForest': {
        'clf__n_estimators': [100, 200],
        'clf__criterion': ['gini', 'entropy'],
        'clf__max_depth': [None, 10, 20],
        'clf__min_samples_split': [2, 5],
        'clf__min_samples_leaf': [1, 2]
    },
    'GradientBoosting': {
        'clf__n_estimators': [100, 200, 300],
        'clf__learning_rate': [0.01, 0.05, 0.1],
        'clf__max_depth': [3, 5],
        'clf__subsample': [0.8, 1.0]
    },
    'AdaBoost': {
    'clf__n_estimators': [50, 100, 200, 300],
    'clf__learning_rate': [0.001, 0.05, 0.1, 0.5, 1],
    },

    'XGBoost': {
        'clf__n_estimators': [100, 200, 300],
        'clf__learning_rate': [0.01, 0.05, 0.1],
        'clf__max_depth': [3, 5, 7],
        'clf__subsample': [0.8, 1.0],
        'clf__colsample_bytree': [0.8, 1.0]
    }
}

In [ ]:
scale_needed = ['SVC']

In [ ]:
def tune_models(X_train, y_train, X_test, y_test, models, param_grids, scale_needed):
    results = {}

    for name, model in models.items():
        print(f"\n🔍 Tuning {name}...")

        # Build pipeline
        if name in scale_needed:
            pipe = Pipeline([
                ('scaler', MinMaxScaler()),
                ('clf', model)
            ])
        else:
            pipe = Pipeline([
                ('clf', model)
            ])

        # For boosting models use only RandomizedSearchCV
        if name in ['GradientBoosting', 'AdaBoost', 'XGBoost']:
            print(f" Using RandomizedSearchCV for {name} (boosting model)")
            search = RandomizedSearchCV(
                estimator=pipe,
                param_distributions=param_grids[name],
                n_iter=15,
                cv=3,
                scoring='f1_macro',
                n_jobs=-1,
                random_state=42,
                verbose=2
            )
        else:
            # For other models use both Grid and Randomized
            print(f"Running GridSearchCV and RandomizedSearchCV for {name}...")

            # GridSearch
            grid = GridSearchCV(pipe, param_grids[name], cv=3,
                                scoring='f1_macro', n_jobs=-1, verbose=1)
            grid.fit(X_train, y_train)
            grid_pred = grid.best_estimator_.predict(X_test)

            # RandomizedSearch
            search = RandomizedSearchCV(pipe, param_distributions=param_grids[name],
                                        n_iter=10, cv=3, scoring='f1_macro',
                                       n_jobs=-1, random_state=42, verbose=1)
            search.fit(X_train, y_train)
            rand_pred = search.best_estimator_.predict(X_test)


            results[name] = {
                'Best Grid Params': grid.best_params_,
                'Grid F1 (CV)': grid.best_score_,
                'Grid Test Report': classification_report(y_test, grid_pred, output_dict=True),
                'Best Random Params': search.best_params_,
                'Random F1 (CV)': search.best_score_,
                'Random Test Report': classification_report(y_test, rand_pred, output_dict=True)
            }

            print(f"{name}: Grid F1={grid.best_score_:.4f}, Random F1={search.best_score_:.4f}")
            continue  # Skip re-running the RandomizedSearch-only block

        # For boosting models (RandomizedSearch only)
        search.fit(X_train, y_train)
        best_model = search.best_estimator_
        preds = best_model.predict(X_test)

        results[name] = {
            'Best Random Params': search.best_params_,
            'Random F1 (CV)': search.best_score_,
            'Random Test Report': classification_report(y_test, preds, output_dict=True)
        }

        print(f"{name}: Random F1={search.best_score_:.4f}")

    return results


In [ ]:
results = tune_models(X_train_selected, y_train_res, X_test_selected, y_test, models, param_grids, scale_needed)


🔍 Tuning SVC...
Running GridSearchCV and RandomizedSearchCV for SVC...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
SVC: Grid F1=0.8519, Random F1=0.8519

🔍 Tuning DecisionTree...
Running GridSearchCV and RandomizedSearchCV for DecisionTree...
Fitting 3 folds for each of 18 candidates, totalling 54 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
DecisionTree: Grid F1=0.8687, Random F1=0.8667

🔍 Tuning RandomForest...
Running GridSearchCV and RandomizedSearchCV for RandomForest...
Fitting 3 folds for each of 48 candidates, totalling 144 fits
Fitting 3 folds for each of 10 candidates, totalling 30 fits
RandomForest: Grid F1=0.9375, Random F1=0.9371

🔍 Tuning GradientBoosting...
 Using RandomizedSearchCV for GradientBoosting (boosting model)
Fitting 3 folds for each of 15 candidates, totalling 45 fits
GradientBoosting: Random F1=0.9235

🔍 Tuning AdaBoost...
 Using RandomizedSearchCV for AdaBoos

In [ ]:
for model, res in results.items():
    print(f"\n {model}:")
    if 'Grid F1 (CV)' in res:
        print(f"  GridSearchCV F1: {res['Grid F1 (CV)']:.4f}")
    print(f"  RandomizedSearchCV F1: {res['Random F1 (CV)']:.4f}")
    print("  Best Params:", res.get('Best Grid Params', res['Best Random Params']))


 SVC:
  GridSearchCV F1: 0.8519
  RandomizedSearchCV F1: 0.8519
  Best Params: {'clf__C': 10, 'clf__gamma': 'scale', 'clf__kernel': 'rbf'}

 DecisionTree:
  GridSearchCV F1: 0.8687
  RandomizedSearchCV F1: 0.8667
  Best Params: {'clf__criterion': 'entropy', 'clf__max_depth': None, 'clf__min_samples_split': 2}

 RandomForest:
  GridSearchCV F1: 0.9375
  RandomizedSearchCV F1: 0.9371
  Best Params: {'clf__criterion': 'gini', 'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 200}

 GradientBoosting:
  RandomizedSearchCV F1: 0.9235
  Best Params: {'clf__subsample': 0.8, 'clf__n_estimators': 300, 'clf__max_depth': 5, 'clf__learning_rate': 0.1}

 AdaBoost:
  RandomizedSearchCV F1: 0.7508
  Best Params: {'clf__n_estimators': 300, 'clf__learning_rate': 1}

 XGBoost:
  RandomizedSearchCV F1: 0.9221
  Best Params: {'clf__subsample': 1.0, 'clf__n_estimators': 200, 'clf__max_depth': 5, 'clf__learning_rate': 0.1, 'clf__colsample_bytree': 0.8}


In [ ]:
svc = SVC(C=10,kernel='rbf',gamma='scale')
svc.fit(X_train_selected,y_train_res)
y_pred = svc.predict(X_test_selected)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.05      0.12      0.07        52
           1       0.18      0.32      0.23       552
           2       0.85      0.72      0.78      3091

    accuracy                           0.65      3695
   macro avg       0.36      0.39      0.36      3695
weighted avg       0.74      0.65      0.69      3695



In [ ]:
adb = AdaBoostClassifier(random_state=42,n_estimators=300,learning_rate=1)
adb.fit(X_train_selected,y_train_res)
y_pred = adb.predict(X_test_selected)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.09      0.21      0.12        52
           1       0.24      0.11      0.15       552
           2       0.85      0.91      0.88      3091

    accuracy                           0.78      3695
   macro avg       0.39      0.41      0.39      3695
weighted avg       0.75      0.78      0.76      3695



So to improve the recall, the next step is using scale_weights of each class and train the model with the original unresampled data.

In [ ]:
# Compute class weights based on training data
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
weights_dict = {cls: w for cls, w in zip(classes, weights)}
print("Computed class weights:", weights_dict)

Computed class weights: {np.int64(0): np.float64(27.11006289308176), np.int64(1): np.float64(2.4128183599216344), np.int64(2): np.float64(0.3923630074640451)}


XGBoost is chosen beacuse its fast computation and handles the imbalanced data.

In [ ]:
# Prepare per-sample weights
sample_weights = np.array([weights_dict[int(y)] for y in y_train])

# Initialize XGBoost model (best params from tuning)
xgb_model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    subsample=1.0,
    colsample_bytree=0.8,
    random_state=42,
    num_class=3
)

# Train with class weights (on original, unresampled data)
xgb_model.fit(X_train, y_train, sample_weight=sample_weights)

# Predict on test set
y_pred = xgb_model.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred, target_names=['Fatal', 'Serious', 'Slight']))

              precision    recall  f1-score   support

       Fatal       0.12      0.29      0.17        52
     Serious       0.24      0.39      0.30       552
      Slight       0.87      0.75      0.81      3091

    accuracy                           0.69      3695
   macro avg       0.41      0.48      0.43      3695
weighted avg       0.77      0.69      0.72      3695



**FEATURE SELECTION BUT WITH NO RESAMPLING**

In [ ]:
importances = xgb_model.feature_importances_
feature_importances = pd.DataFrame({
    'Feature': df.drop(['Accident_severity'],axis=1).columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
print(feature_importances)

selector = SelectFromModel(xgb_model,threshold=0.026934,prefit=True)
selected_indices = selector.get_support(indices=True)
selected_features = df.drop(['Accident_severity'], axis=1).columns[selected_indices]
print("Selected Features:\n",selected_features)
print("No. of features selected:",len(selected_features))

X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

X_train_selected = pd.DataFrame(X_train_selected, columns=selected_features)
X_test_selected = pd.DataFrame(X_test_selected, columns=selected_features)

                        Feature  Importance
19  Number_of_vehicles_involved    0.072391
20         Number_of_casualties    0.071561
16             Light_conditions    0.048659
1            Age_band_of_driver    0.043292
0                   Day_of_week    0.042191
13            Types_of_Junction    0.039312
29            Cause_of_accident    0.037578
21             Vehicle_movement    0.035871
25            Casualty_severity    0.035543
2                 Sex_of_driver    0.034986
5            Driving_experience    0.034587
30                  Time_of_Day    0.034114
22               Casualty_class    0.032732
28          Pedestrian_movement    0.032029
11             Lanes_or_Medians    0.031504
7              Owner_of_vehicle    0.030591
14            Road_surface_type    0.029418
3             Educational_level    0.028448
18            Type_of_collision    0.028124
10        Area_accident_occured    0.027437
26            Work_of_casuality    0.026933
6               Type_of_vehicle 

In [ ]:
xgb_model.fit(X_train_selected, y_train, sample_weight=sample_weights)

# Predict on test set
y_pred = xgb_model.predict(X_test_selected)

# Evaluate
print(classification_report(y_test, y_pred, target_names=['Fatal', 'Serious', 'Slight']))

              precision    recall  f1-score   support

       Fatal       0.13      0.33      0.18        52
     Serious       0.22      0.39      0.28       552
      Slight       0.87      0.72      0.79      3091

    accuracy                           0.67      3695
   macro avg       0.40      0.48      0.42      3695
weighted avg       0.76      0.67      0.70      3695



In [ ]:
selected_scaler = MinMaxScaler()
selected_scaler.fit(X_train_selected)

# Save the scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(selected_scaler, f)

To improve the recall and precision better, we use a two-stage approach to train the model with the imbalanced dataset.

**STAGE 1: Training the 1st model with Fatal vs Non-Fatal**

In [ ]:
from sklearn.metrics import precision_recall_curve, recall_score, classification_report

# Build binary labels for stage 1
y_train_bin = (y_train == 0).astype(int)   # 1 if Fatal, 0 otherwise
y_test_bin  = (y_test == 0).astype(int)

# Train binary XGB (tweak params as per your prior tuning)
xgb_bin = XGBClassifier(
    objective='binary:logistic',
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    colsample_bytree=0.8,
    subsample=1.0,
    random_state=42
)

# use per-sample weights: for binary stage, amplify Fatal more if desired
# One option: use the earlier class weights for original labels mapped to binary:
sample_weights_bin = np.array([weights_dict[int(y)] for y in y_train])  # re-uses balanced weights

xgb_bin.fit(X_train_selected, y_train_bin, sample_weight=sample_weights_bin)

# Get probabilities on validation set and find threshold that gives high recall on Fatal
probs_val = xgb_bin.predict_proba(X_test_selected)[:,1]  # prob of Fatal
prec, rec, thresholds = precision_recall_curve(y_test_bin, probs_val)

# Choose threshold for high recall (e.g., maximize recall while keeping precision >= min_prec)
min_precision_required = 0.05  # set small if you accept many false positives; tune this
candidates = [(t, p, r) for t, p, r in zip(np.append(thresholds, 1.0), prec, rec) if p >= min_precision_required]
if len(candidates) == 0:
    # fallback: choose threshold that gives highest recall
    best_idx = np.argmax(rec)
    best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 1.0
else:
    # choose candidate with highest recall among those passing min precision
    best_threshold, best_prec, best_rec = max(candidates, key=lambda x: x[2])

print("Chosen threshold for Fatal:", best_threshold)

# Evaluate on test set with chosen threshold
probs_test = xgb_bin.predict_proba(X_test_selected)[:,1]
pred_bin = (probs_test >= best_threshold).astype(int)
print("Binary Fatal detection report:")
print(classification_report(y_test_bin, pred_bin, target_names=['Not-Fatal','Fatal']))

Chosen threshold for Fatal: 0.07830092310905457
Binary Fatal detection report:
              precision    recall  f1-score   support

   Not-Fatal       0.99      0.83      0.91      3643
       Fatal       0.05      0.62      0.09        52

    accuracy                           0.83      3695
   macro avg       0.52      0.72      0.50      3695
weighted avg       0.98      0.83      0.89      3695



In [ ]:
pickle.dump(xgb_bin,open('xgb_fatal_notfatal.pkl','wb'))

**Stage 2: Train the 2nd model with 'Serious Injury' vs 'Slight Injury'**

In [ ]:
# Reset indices
X_train_selected = X_train_selected.reset_index(drop=True)
X_test_selected = X_test_selected.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Keep only the non-fatal samples (exclude Fatal = 0)
X_train_nonfatal = X_train_selected[y_train != 0]
X_test_nonfatal  = X_test_selected[y_test != 0]
y_train_nonfatal = y_train[y_train != 0]
y_test_nonfatal  = y_test[y_test != 0]

# Re-map classes for binary classification
# Serious (1) → 1, Slight (2) → 0
y_train_nonfatal_bin = (y_train_nonfatal == 1).astype(int)
y_test_nonfatal_bin  = (y_test_nonfatal == 1).astype(int)

# Compute class weights to handle imbalance
class_weights_nonfatal = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_nonfatal_bin),
    y=y_train_nonfatal_bin
)
weights_dict_nonfatal = dict(enumerate(class_weights_nonfatal))
sample_weights_nonfatal = np.array([weights_dict_nonfatal[int(y)] for y in y_train_nonfatal_bin])

# Define the Stage 2 model
xgb_multi = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    subsample=1.0,
    colsample_bytree=0.8,
    random_state=42
)

# Train the Stage 2 model
xgb_multi.fit(X_train_nonfatal, y_train_nonfatal_bin, sample_weight=sample_weights_nonfatal)

# Predict and evaluate
y_pred_nonfatal = xgb_multi.predict(X_test_nonfatal)


if len(y_pred_nonfatal.shape) > 1:
    y_pred_nonfatal = np.argmax(y_pred_nonfatal, axis=1)

print("Stage 2 Model (Serious vs Slight)")
print("Accuracy:", accuracy_score(y_test_nonfatal_bin, y_pred_nonfatal))
print("\nClassification Report:\n", classification_report(
    y_test_nonfatal_bin, y_pred_nonfatal,
    target_names=["Slight Injury", "Serious Injury"]
))

Stage 2 Model (Serious vs Slight)
Accuracy: 0.706286027998902

Classification Report:
                 precision    recall  f1-score   support

 Slight Injury       0.88      0.76      0.81      3091
Serious Injury       0.23      0.41      0.30       552

      accuracy                           0.71      3643
     macro avg       0.56      0.59      0.56      3643
  weighted avg       0.78      0.71      0.74      3643



In [ ]:
pickle.dump(xgb_multi,open('xgb_serious_slight.pkl','wb'))

In [ ]:
# Testing the prediction with testing data
def predict_severity(X_input):
    # Ensure input is in the correct shape for prediction
    if isinstance(X_input, np.ndarray):
        X_input = X_input.reshape(1, -1)
        X_input = pd.DataFrame(X_input, columns=X_train.columns)
    elif isinstance(X_input, pd.Series):
        X_input = X_input.to_frame().T  # convert single row Series → DataFrame
    elif isinstance(X_input, pd.DataFrame):
        pass
    else:
        print("Input must be a numpy array, pandas Series, or pandas DataFrame.")

    # Apply feature selection and convert back to DataFrame
    X_input_selected_array = selector.transform(X_input)
    X_input_selected_df = pd.DataFrame(X_input_selected_array, columns=selected_features)

    # Stage 1: Fatal vs Non-Fatal
    fatal_prob = xgb_bin.predict_proba(X_input_selected_df)[:, 1][0]  # Probability of Fatal
    fatal_pred = (fatal_prob >= 0.07830092310905457)  # your chosen threshold

    if fatal_pred:
        return "Fatal Injury"

    # Stage 2: Serious vs Slight (only if not fatal)
    stage2_pred = xgb_multi.predict(X_input_selected_df)[0]
    # stage2_pred = 1 → Serious, 0 → Slight
    if stage2_pred == 1:
        return "Serious Injury"
    else:
        return "Slight Injury"

In [ ]:
# Suppose you have a single test sample:
sample = X_test_selected.iloc[9]
predicted_label = predict_severity(sample)
print("Predicted Accident Severity:", predicted_label)

Predicted Accident Severity: Fatal Injury


In [ ]:
pickle.dump(encoders,open('encoders.pkl','wb'))